In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
plt.rcParams['font.size'] = 22
import sys
import seaborn as sns
import xarray as xr

In [2]:
if 'win' in sys.platform:
    path = "E:/OneDrive/PhD/PhD/Data/Hintereisferner/COSIPY/MiscTests/"
    tsla = pd.read_csv("E:/OneDrive/PhD/PhD/Data/Hintereisferner/Climate/snowlines/HEF-snowlines-1999-2010_manual_filtered.csv", parse_dates=True, index_col="LS_DATE")
else:
    path = "/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/COSIPY/MiscTests/"
    tsla = pd.read_csv("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Climate/snowlines/HEF-snowlines-1999-2010_manual_filtered.csv", parse_dates=True, index_col="LS_DATE")
snowlinepath = path+"manual_sens_snowlines/"


In [ ]:
## Load reference MB and snowlines
time_start_dt = pd.to_datetime("2000-01-01") #config starts with spinup - need to add 1year
time_end_dt = pd.to_datetime("2009-12-31")
print("Start date:", time_start_dt)
print("End date:", time_end_dt)
tsla = tsla.loc[time_start_dt:time_end_dt]
#Normalize standard deviation if necessary
tsla['SC_stdev'] = (tsla['SC_stdev']) / (tsla['glacier_DEM_max'] - tsla['glacier_DEM_min'])

thres_unc = (20) / (tsla['glacier_DEM_max'].iloc[0] - tsla['glacier_DEM_min'].iloc[0])
print(thres_unc)

## Set observational uncertainty where smaller to atleast model resolution (20m) and where larger keep it
sc_norm = np.where(tsla['SC_stdev'] < thres_unc, thres_unc, tsla['SC_stdev'])
tsla['SC_stdev'] = sc_norm

tsla

In [ ]:
## Load MB data
rgi_id = "RGI60-11.00897"
if 'win' in sys.platform:
    geod_mb = pd.read_csv("E:/OneDrive/PhD/PhD/Data/Hugonnet_21_MB/dh_11_rgi60_pergla_rates.csv")
else:
    geod_mb = pd.read_csv("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hugonnet_21_MB/dh_11_rgi60_pergla_rates.csv")
geod_mb = geod_mb.loc[geod_mb['rgiid'] == rgi_id]
geod_mb = geod_mb.loc[geod_mb['period'] == "2000-01-01_2010-01-01"]
geod_mb = geod_mb[['dmdtda', 'err_dmdtda']]
print(geod_mb)

In [ ]:
## Prepare loglike calculations - snowline metrics: Maximum TSLA + Std and normal timeseries? Rate of change?
tsla['year'] = tsla.index.year

max_tsl_data = tsla.loc[tsla.groupby('year')["TSL_normalized"].idxmax()]
#max_tsl_data['SC_stdev'] = max_tsl_data['SC_stdev']

max_tsl_with_uncertainty = max_tsl_data[["TSL_normalized", 'SC_stdev']]

matching_row_numbers = [tsla.index.get_loc(date) for date in max_tsl_with_uncertainty.index if date in tsla.index]
print("Matching row numbers in df1:", matching_row_numbers)

max_tsl_with_uncertainty['rownumber'] = matching_row_numbers 
print(max_tsl_with_uncertainty)

In [6]:
def loglike_tsla_func(sim_tsla, eval_tsla, sigma_tsla):
    loglike_tsla = -0.5 * np.sum(np.log(2 * np.pi * sigma_tsla**2) + ((eval_tsla-sim_tsla)**2 / sigma_tsla**2))
    avg_loglike_tsla = loglike_tsla / len(eval_tsla)
    return avg_loglike_tsla

def loglike_mb_func(sim_mb, eval_mb, sigma_mb):
    loglike_mb = -0.5 * (np.log(2 * np.pi * sigma_mb**2) + ( ((eval_mb-sim_mb)**2) / sigma_mb**2))
    return loglike_mb



In [ ]:
df = pd.read_csv(path+"localsens_1D20m_1999_2010.csv", index_col=0)
df

In [ ]:
## Load ALBEDO observations 
if 'win' in sys.platform:
    alb_obs_data = xr.open_dataset(r"E:\OneDrive\PhD\PhD\Data\Hintereisferner\Climate\HEF_processed_HRZ-30CC-filter_albedos.nc")
else:
    alb_obs_data = xr.open_dataset("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Climate/HEF_processed_HRZ-30CC-filter_albedos.nc")
#has nans where no glacier -> can build glacier-wide mean albedo for additional logp
alb_obs_data = alb_obs_data.sortby("time")
alb_obs_data

In [ ]:
## Load corresponding albedo files
sigma_alb = alb_obs_data['sigma_albedo'] #alb_obs_std.albedo / np.sqrt(alb_obs_sample_size.albedo) #Standard Error of the Mean assuming no autocorrelation (which is faulty but just a first order approximation), error super small
eval_alb = alb_obs_data['median_albedo']

list_logp_alb = []
list_sim_alb = []

for i,r in df.iterrows():
    if i % 100 == 0:
        print(f"Processing file {i}/2500")
    rrr_factor = round(r['rrr_factor'],4)
    alb_ice = round(r['alb_ice'],4)
    alb_snow = round(r['alb_snow'],4)
    alb_firn = round(r['alb_firn'],4)
    alb_aging = round(r['albedo_aging'],4)
    alb_depth = round(r['albedo_depth'],4)
    roughness_ice = round(r['roughness_ice'], 4)
    roughness_firn = round(r['roughness_firn'], 4)
    roughness_snow = round(r['roughness_fresh_snow'], 4)
    roughness_aging = round(r['aging_factor_roughness'], 6)
    filename = f"HEF_COSMO_1D20m_1999_2010_HORAYZON_IntpPRES_localsens_19990101-20091231_RRR-{rrr_factor}_{alb_snow}_{alb_ice}_{alb_firn}_{alb_aging}_{alb_depth}_{roughness_snow}_{roughness_ice}_{roughness_firn}_{roughness_aging}_num2.nc"
    if 'win' in sys.platform:
        sim_alb = xr.open_dataset("E:/OneDrive/PhD/PhD/Data/Hintereisferner/Output/albedo_files/sens_test/"+\
            filename)
    else:
        sim_alb = xr.open_dataset("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Output/albedo_files/sens_test/"+\
            filename)

    
    #sort by time 
    sim_alb = sim_alb['ALBEDO_weighted'].sortby("time")
    sim_alb = sim_alb.sel(time=alb_obs_data.time)
        
    ## compute logp albedo for file
    list_sim_alb.append(sim_alb.data)
    logp_alb_all = -0.5 * np.sum(np.log(2 * np.pi * sigma_alb**2) + ((sim_alb.data-eval_alb)**2 / sigma_alb**2))
    avg_logp_alb = logp_alb_all / len(sim_alb)
    list_logp_alb.append(avg_logp_alb.item())


In [10]:
alb_logp = pd.Series(list_logp_alb)
df['logpALB'] = alb_logp

In [ ]:
## Calc Log Likelihoods
mb_logp = df[['mb']].apply(loglike_mb_func, eval_mb = geod_mb['dmdtda'].values, sigma_mb= geod_mb['err_dmdtda'].values)
df['logpMB'] = mb_logp

df

In [ ]:
## Compute logp snowlines - use saved sims instead of extra files! Extra files are missing 2 snowlines
modtsls = df.iloc[:,13:71].transpose()
print(modtsls)

#max_modtsls = modtsls.iloc[max_tsl_with_uncertainty['rownumber']]
#print(max_modtsls)


'''
## Data got deleted - skip computation.
tsl_logps = []
indexes = []
for i,r in df.iterrows():
    rrrfactor = r['rrr_factor']
    albsnow = r['alb_snow']
    albice = r['alb_ice']
    albfirn = r['alb_firn']
    aaging = r['albedo_aging']
    adepth = r['albedo_depth']
    zsnow = r['roughness_fresh_snow']
    zice = r['roughness_ice']
    zfirn = r['roughness_firn']
    zage = r['aging_factor_roughness']
    filename = f"tsla_hef_cosmo_1d20m_1999_2010_horayzon_mansensfull_19990101-20091231_rrr-{rrrfactor}_{albsnow}_{albice}_{albfirn}_{aaging}_{adepth}_{zsnow}_{zice}_{zfirn}_{round(zage,4)}_num2.csv"
    indexes.append(filename)
    if cond == "fullprior":
        tsl = pd.read_csv(snowlinepath+"fullprior/"+filename, index_col="time", parse_dates=True)
    else:
        tsl = pd.read_csv(snowlinepath+"lowsnow/"+filename, index_col="time", parse_dates=True)
    tsl_sub = tsl.loc[tsl.index.isin(tsla.index)]
    tsl_logp = tsl_sub[['Med_TSL']].apply(loglike_tsla_func, eval_tsla =tsla['TSL_normalized'].values, sigma_tsla= tsla['SC_stdev'].values, axis=0)
    tsl_logps.append(tsl_logp.item())
    
    del filename 
'''
#tsl_res = modtsls.apply(mae, obs_vals = tsla_synth['Med_TSL'].values, obs_sigma=tsla_synth['Std_TSL'].values, axis=0) #
tsl_logp = modtsls.apply(loglike_tsla_func, eval_tsla =tsla['TSL_normalized'].values, sigma_tsla= tsla['SC_stdev'].values, axis=0)
print(tsl_logp)
df['logpTSL'] = tsl_logp
#tslmax_logp = max_modtsls.apply(loglike_tsla_func, eval_tsla =max_tsl_with_uncertainty['TSL_normalized'].values, sigma_tsla= max_tsl_with_uncertainty['SC_stdev'].values, axis=0)
#print(tsl_logp)
 

In [ ]:
## Load EB terms (similar like we loaded albedo)
list_sim_ablme = []
list_sim_ablann = []

for i,r in df.iterrows():
    if i % 100 == 0:
        print(f"Processing file {i}/2500")
    rrr_factor = round(r['rrr_factor'],4)
    alb_ice = round(r['alb_ice'],4)
    alb_snow = round(r['alb_snow'],4)
    alb_firn = round(r['alb_firn'],4)
    alb_aging = round(r['albedo_aging'],4)
    alb_depth = round(r['albedo_depth'],4)
    roughness_ice = round(r['roughness_ice'], 4)
    roughness_firn = round(r['roughness_firn'], 4)
    roughness_snow = round(r['roughness_fresh_snow'], 4)
    roughness_aging = round(r['aging_factor_roughness'], 6)
    filename = f"HEF_COSMO_1D20m_1999_2010_HORAYZON_IntpPRES_localsens_19990101-20091231_RRR-{rrr_factor}_{alb_snow}_{alb_ice}_{alb_firn}_{alb_aging}_{alb_depth}_{roughness_snow}_{roughness_ice}_{roughness_firn}_{roughness_aging}_num2.nc"
    if 'win' in sys.platform:
        sim_ds = xr.open_dataset("E:/OneDrive/PhD/PhD/Data/Hintereisferner/Output/sens_me/"+\
            filename)
    else:
        sim_ds = xr.open_dataset("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Output/sens_me/"+\
            filename)

    
    #mean over hydroyear
    sim_ds = sim_ds.mean(dim='hydroyear') 
        
    ## compute logp albedo for file
    list_sim_ablme.append(sim_ds['ME_abl'].item())
    list_sim_ablann.append(sim_ds['ME_ann'].item())

In [ ]:
df['MEabl'] = list_sim_ablme
df['MEann'] = list_sim_ablann
df

In [15]:
dic_label = {'rrr_factor': r'$p_{f}$', 'alb_ice': r'$\alpha_{ice}$', 'alb_snow': r'$\alpha_{fs}$','alb_firn': r'$\alpha_{firn}$', 'albedo_aging': r'$\alpha_{aging}$',
                'albedo_depth': r'$\alpha_{depth}$','roughness_ice': r'$z0_{ice}$','roughness_firn': r'$z0_{firn}$', 'roughness_fresh_snow': r'$z0_{fs}$', 'aging_factor_roughness': r'$z0_{aging}$'}

lookup_zero = {'rrr_factor': 1.25,
               'alb_ice': 0.28,
               'alb_snow': 0.865,
               'alb_firn': 0.605,
               'albedo_aging': 13.00,
               'albedo_depth': 8.00,
               'roughness_fresh_snow': 0.81,
               'roughness_ice': 10.35,
               'roughness_firn': 4.05,
               'aging_factor_roughness': 0.0026}

## come up with a function to calculate relevant values, extract relevant data
def process_data(df, var, ref_row=0):
    df_copy = df.sort_values(by=var)
    df_copy['Std'] = (df_copy[var] - df_copy[var].min()) / (df_copy[var].max() - df_copy[var].min())
    df_copy['PercMB'] = df_copy['mb'] - df_copy.loc[ref_row]['mb']
    
    return df_copy

## parse into plot function to call and create figure
def create_plot_object(df, var, xticks, yticks,  axis, ylabel=True, ylabel2=True, color="steelblue"):
    
    # Define the color mapping for specific precipitation factors
    if var == "rrr_factor":
        axis.plot(df['Std'], df['PercMB'], marker="o", linestyle="--", color="slategrey")
        color_map = {0.5: 'steelblue', 1.25: 'lightseagreen', 2.0: 'peru'}
        # Plot specific precipitation factors with assigned colors
        for pf, color in color_map.items():
            subset = df.loc[df['rrr_factor'] == pf]
            axis.plot(subset['Std'], subset['PercMB'], marker='o', color=color)
    else:
        axis.plot(df['Std'], df['PercMB'], marker="o", linestyle="--", color=color)
    try:
        #get 0 place
        zero_point = df.loc[df['PercMB'] == 0.0]
        #print(zero_point)
        axis.axhline(y=zero_point['PercMB'].values, linestyle="--", color="black", zorder=-1, alpha=0.7)
        axis.axvline(x=zero_point['Std'].values, linestyle="--", color="black", zorder=-1, alpha=0.7)
    except:
        std_ref = lookup_zero[var]
        zero_point = df.loc[df[var] == std_ref]
        print(zero_point)
        axis.axhline(y=zero_point['PercMB'].values, linestyle="--", color="black", zorder=-1, alpha=0.7)
        axis.axvline(x=zero_point['Std'].values, linestyle="--", color="black", zorder=-1, alpha=0.7)
    
    #ax.axhline(y=-1.0425, color="black", label="Hugonnet et al. (2021)")
    #ax.fill_between(df['Std'], y1=-1.0425-0.261, y2=-1.0425+0.261, color="grey", alpha=0.5)
    #-1.0425       0.261
    #ax2 = axis.twinx()
    #ax2.plot(df['Std'], df['logpTSL'], marker=".", linestyle="--", color="darkgreen")
    if ylabel:
        axis.set_ylabel(r"$\Delta$ MB (m w.e.)")
    else:
        axis.set_ylabel("")
    
    #if ylabel2:
    #    ax2.set_ylabel("Logp. TSLA")
    #else:
    #    ax2.set_ylabel("")
    axis.yaxis.label.set_color('slategrey')
    axis.tick_params(axis='y', colors='slategrey')
    
    #ax2.yaxis.label.set_color('darkgreen')
    #ax2.tick_params(axis='y', colors='darkgreen')

    # Add a secondary x-axis below the primary x-axis
    def custom_ticks(x):
        #convert back to original values
        orig_max = df[var].max()
        orig_min = df[var].min()
        return x * (orig_max - orig_min) + orig_min  # Define your custom transformation for tick labels

    #if cond == "fullprior":
    ax2x = axis.secondary_xaxis(-0.15, functions=(custom_ticks, custom_ticks))
    #else:
    #ax2x = axis.secondary_xaxis(-0.23, functions=(custom_ticks, custom_ticks))
    ax2x.set_xticks(xticks)
    if var == "aging_factor_roughness":
        ax2x.set_xticklabels([round(x,5) for x in ax2x.get_xticks()], rotation=35)
    else:
        ax2x.set_xticklabels([round(x,2) for x in ax2x.get_xticks()], rotation=35)
    ax2x.set_xlabel("")
    
    axis.set_title(dic_label[var])
    #ax.grid()
    axis.set_xlim(0, 1)
    #axis.set_yticks(yticks)
    

## function needs to create axes argument so we can paste them together to create a single figure 

In [ ]:
prec_df = pd.concat([df.iloc[0:10], df.loc[df['rrr_factor'] == 1.25].iloc[[0]]], ignore_index=True).sort_values(by='rrr_factor').reset_index(drop=True)
prec_df


In [17]:
sub2 = df.loc[df['rrr_factor'] == 1.25].reset_index(drop=True)

In [ ]:
plt.rcParams['font.size'] =  22
fig, ax = plt.subplots(2,5, figsize=(20,12), dpi=300, sharey=True)
## Prcp factor all versions and first for prcp factor = 0.5
sub = df.loc[df['rrr_factor'] == 0.5].reset_index(drop=True)
create_plot_object(process_data(prec_df, var="rrr_factor", ref_row=5), "rrr_factor", np.arange(0,2+0.5,0.5), np.arange(-6, 4+2, 2), axis=ax[0,0], ylabel=True, ylabel2=False)
create_plot_object(process_data(sub.iloc[[0] + list(range(1,11))], var="alb_ice", ref_row=0), "alb_ice", np.arange(0.1,0.46+0.03*3,0.03*3), np.arange(-0.06, 0.06+0.04, 0.04), axis=ax[0,1], ylabel=False, ylabel2=False)
create_plot_object(process_data(sub.iloc[[0] + list(range(11,21))], var="alb_snow", ref_row=0), "alb_snow", np.arange(0.75,0.98+0.025556*2,0.025556*2), np.arange(-1.5, 1.5+0.5, 0.5), axis=ax[0,2], ylabel=False, ylabel2=True)
create_plot_object(process_data(sub.iloc[[0] + list(range(21,31))], var="alb_firn", ref_row=0), "alb_firn", np.arange(0.46,0.75+0.03222*2,0.03222*2), np.arange(-0.45, 0.45+0.15, 0.15), axis=ax[0,3], ylabel=False, ylabel2=False)
create_plot_object(process_data(sub.iloc[[0] + list(range(31,41))], var="albedo_aging", ref_row=0), "albedo_aging", np.arange(1, 25+2.666666*2,2.666666*2), np.arange(-3, 0.5+0.9, 0.9), axis=ax[0,4], ylabel=False, ylabel2=False)
create_plot_object(process_data(sub.iloc[[0] + list(range(41,51))], var="albedo_depth", ref_row=0), "albedo_depth", np.arange(1,15+1.555555*2,1.555555*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,0], ylabel=True, ylabel2=True)
create_plot_object(process_data(sub.iloc[[0] + list(range(61,71))], var="roughness_ice", ref_row=0), "roughness_ice", np.arange(0.7,20.0+2.1444444*2,2.1444444*2), np.arange(-0.015, 0.005+0.005, 0.005), axis=ax[1,1], ylabel=False, ylabel2=True)
create_plot_object(process_data(sub.iloc[[0] + list(range(51,61))], var="roughness_fresh_snow", ref_row=0), "roughness_fresh_snow", np.arange(0.02,1.6+0.17555*2,0.17555*2), np.arange(-0.6, 0.4+0.1, 0.1), axis=ax[1,2], ylabel=False, ylabel2=True)
create_plot_object(process_data(sub.iloc[[0] + list(range(71,81))], var="roughness_firn", ref_row=0), "roughness_firn", np.arange(1.6,6.5+0.544444*2,0.544444*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,3], ylabel=False, ylabel2=True)
create_plot_object(process_data(sub.iloc[[0] + list(range(81,91))], var="aging_factor_roughness", ref_row=0), "aging_factor_roughness", np.arange(0.0013,0.0039+0.00026*2,0.00026*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,4], ylabel=False, ylabel2=True)
#
sub2 = df.loc[df['rrr_factor'] == 1.25].reset_index(drop=True)
#create_plot_object(process_data(prec_df, var="rrr_factor", ref_row=5), "rrr_factor", np.arange(0,2+0.5,0.5), np.arange(-6, 4+2, 2), axis=ax[0,0], ylabel=True, ylabel2=False, color="lightseagreen")
create_plot_object(process_data(sub2.iloc[[0] + list(range(1,11))], var="alb_ice", ref_row=0), "alb_ice", np.arange(0.1,0.46+0.03*3,0.03*3), np.arange(-0.06, 0.06+0.04, 0.04), axis=ax[0,1], ylabel=False, ylabel2=False, color="lightseagreen")
create_plot_object(process_data(sub2.iloc[[0] + list(range(11,21))], var="alb_snow", ref_row=0), "alb_snow", np.arange(0.75,0.98+0.025556*2,0.025556*2), np.arange(-1.5, 1.5+0.5, 0.5), axis=ax[0,2], ylabel=False, ylabel2=True, color="lightseagreen")
create_plot_object(process_data(sub2.iloc[[0] + list(range(21,31))], var="alb_firn", ref_row=0), "alb_firn", np.arange(0.46,0.75+0.03222*2,0.03222*2), np.arange(-0.45, 0.45+0.15, 0.15), axis=ax[0,3], ylabel=False, ylabel2=False, color="lightseagreen")
create_plot_object(process_data(sub2.iloc[[0] + list(range(31,41))], var="albedo_aging", ref_row=0), "albedo_aging", np.arange(1, 25+2.666666*2,2.666666*2), np.arange(-3, 0.5+0.9, 0.9), axis=ax[0,4], ylabel=False, ylabel2=False, color="lightseagreen")
create_plot_object(process_data(sub2.iloc[[0] + list(range(41,51))], var="albedo_depth", ref_row=0), "albedo_depth", np.arange(1,15+1.555555*2,1.555555*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,0], ylabel=True, ylabel2=True, color="lightseagreen")
create_plot_object(process_data(sub2.iloc[[0] + list(range(61,71))], var="roughness_ice", ref_row=0), "roughness_ice", np.arange(0.7,20.0+2.1444444*2,2.1444444*2), np.arange(-0.015, 0.005+0.005, 0.005), axis=ax[1,1], ylabel=False, ylabel2=True, color="lightseagreen")
create_plot_object(process_data(sub2.iloc[[0] + list(range(51,61))], var="roughness_fresh_snow", ref_row=0), "roughness_fresh_snow", np.arange(0.02,1.6+0.17555*2,0.17555*2), np.arange(-0.6, 0.4+0.1, 0.1), axis=ax[1,2], ylabel=False, ylabel2=True, color="lightseagreen")
create_plot_object(process_data(sub2.iloc[[0] + list(range(71,81))], var="roughness_firn", ref_row=0), "roughness_firn", np.arange(1.6,6.5+0.544444*2,0.544444*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,3], ylabel=False, ylabel2=True, color="lightseagreen")
create_plot_object(process_data(sub2.iloc[[0] + list(range(81,91))], var="aging_factor_roughness", ref_row=0), "aging_factor_roughness", np.arange(0.0013,0.0039+0.00026*2,0.00026*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,4], ylabel=False, ylabel2=True, color="lightseagreen")
#
sub3 = df.loc[df['rrr_factor'] == 2.0].reset_index(drop=True)
#create_plot_object(process_data(prec_df, var="rrr_factor", ref_row=5), "rrr_factor", np.arange(0,2+0.5,0.5), np.arange(-6, 4+2, 2), axis=ax[0,0], ylabel=True, ylabel2=False, color="lightseagreen")
create_plot_object(process_data(sub3.iloc[[0] + list(range(1,11))], var="alb_ice", ref_row=0), "alb_ice", np.arange(0.1,0.46+0.03*3,0.03*3), np.arange(-0.06, 0.06+0.04, 0.04), axis=ax[0,1], ylabel=False, ylabel2=False, color="peru")
create_plot_object(process_data(sub3.iloc[[0] + list(range(11,21))], var="alb_snow", ref_row=0), "alb_snow", np.arange(0.75,0.98+0.025556*2,0.025556*2), np.arange(-1.5, 1.5+0.5, 0.5), axis=ax[0,2], ylabel=False, ylabel2=True, color="peru")
create_plot_object(process_data(sub3.iloc[[0] + list(range(21,31))], var="alb_firn", ref_row=0), "alb_firn", np.arange(0.46,0.75+0.03222*2,0.03222*2), np.arange(-0.45, 0.45+0.15, 0.15), axis=ax[0,3], ylabel=False, ylabel2=False, color="peru")
create_plot_object(process_data(sub3.iloc[[0] + list(range(31,41))], var="albedo_aging", ref_row=0), "albedo_aging", np.arange(1, 25+2.666666*2,2.666666*2), np.arange(-3, 0.5+0.9, 0.9), axis=ax[0,4], ylabel=False, ylabel2=False, color="peru")
create_plot_object(process_data(sub3.iloc[[0] + list(range(41,51))], var="albedo_depth", ref_row=0), "albedo_depth", np.arange(1,15+1.555555*2,1.555555*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,0], ylabel=True, ylabel2=True, color="peru")
create_plot_object(process_data(sub3.iloc[[0] + list(range(61,71))], var="roughness_ice", ref_row=0), "roughness_ice", np.arange(0.7,20.0+2.1444444*2,2.1444444*2), np.arange(-0.015, 0.005+0.005, 0.005), axis=ax[1,1], ylabel=False, ylabel2=True, color="peru")
create_plot_object(process_data(sub3.iloc[[0] + list(range(51,61))], var="roughness_fresh_snow", ref_row=0), "roughness_fresh_snow", np.arange(0.02,1.6+0.17555*2,0.17555*2), np.arange(-0.6, 0.4+0.1, 0.1), axis=ax[1,2], ylabel=False, ylabel2=True, color="peru")
create_plot_object(process_data(sub3.iloc[[0] + list(range(71,81))], var="roughness_firn", ref_row=0), "roughness_firn", np.arange(1.6,6.5+0.544444*2,0.544444*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,3], ylabel=False, ylabel2=True, color="peru")
create_plot_object(process_data(sub3.iloc[[0] + list(range(81,91))], var="aging_factor_roughness", ref_row=0), "aging_factor_roughness", np.arange(0.0013,0.0039+0.00026*2,0.00026*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,4], ylabel=False, ylabel2=True, color="peru")

fig.subplots_adjust(hspace=0.45, wspace=0.3)  # Adjust as needed
#fig.tight_layout()
list_vals = [1.25, 0.28, 0.865, 0.605, 13.00, 8.00, 10.35, 0.81, 4.05, 0.0026]
i = 0
for axis in [ax[0,0], ax[0,1], ax[0,2], ax[0,3], ax[0,4]]:
    if axis == ax[0,1]:
        axis.annotate(list_vals[i], xy=(0.78, 0.01), xycoords="axes fraction", alpha=0.7, fontsize=16)
    elif axis == ax[0,0]:
        axis.annotate(list_vals[i], xy=(0.78, 0.01), xycoords="axes fraction", alpha=0.7, fontsize=16)
    elif axis == ax[0,4]:
        axis.annotate(list_vals[i], xy=(0.78, 0.01), xycoords="axes fraction", alpha=0.7, fontsize=16)
    else:
        axis.annotate(list_vals[i], xy=(0.71, 0.01), xycoords="axes fraction", alpha=0.7, fontsize=16)
    i+= 1
for axis in [ax[1,0], ax[1,1], ax[1,2], ax[1,3], ax[1,4]]:
    if axis == ax[1,1]:
        axis.annotate(list_vals[i], xy=(0.70, 0.95), xycoords="axes fraction", alpha=0.7, fontsize=16)
    elif axis == ax[1,0]:
        axis.annotate(list_vals[i], xy=(0.823, 0.95), xycoords="axes fraction", alpha=0.7, fontsize=16)
    elif axis == ax[1,4]:
        axis.annotate(list_vals[i], xy=(0.632, 0.95), xycoords="axes fraction", alpha=0.7, fontsize=16)
    else:
        axis.annotate(list_vals[i], xy=(0.767, 0.95), xycoords="axes fraction", alpha=0.7, fontsize=16)
    i+= 1
# Define precipitation factors and corresponding colors
precip_factors = [0.5, 1.25, 2.0]
colors = ["steelblue", "lightseagreen", "peru"]
linestyle = "--"
marker = "o"

# Create custom legend handles
legend_handles = [
    mlines.Line2D([], [], color=color, linestyle=linestyle, marker=marker, label=f"prcp. factor {factor}")
    for color, factor in zip(colors, precip_factors)
]

# Add figure legend below the plot
fig.legend(handles=legend_handles, loc="upper center", bbox_to_anchor=(0.5, -0.05), ncol=len(precip_factors))

# Adjust spacing to bring the legend closer to the plot
fig.subplots_adjust(bottom=0.07)  # Decrease bottom margin
#if 'win' in sys.platform:
#    plt.savefig("E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/local_sensitivites_mb.png", bbox_inches="tight")
#else:
#    plt.savefig("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/local_sensitivites_mb.png", bbox_inches="tight")

In [ ]:
# set up helper dictionaries
dic_label = {
    'rrr_factor': r'$p_{f}$', 
    'alb_ice': r'$\alpha_{ice}$', 
    'alb_snow': r'$\alpha_{fs}$',
    'alb_firn': r'$\alpha_{firn}$', 
    'albedo_aging': r'$\alpha_{aging}$',
    'albedo_depth': r'$\alpha_{depth}$',
    'roughness_ice': r'$z0_{ice}$',
    'roughness_firn': r'$z0_{firn}$', 
    'roughness_fresh_snow': r'$z0_{fs}$', 
    'aging_factor_roughness': r'$z0_{aging}$'
}

lookup_zero = {
    'rrr_factor': 1.25,
    'alb_ice': 0.28,
    'alb_snow': 0.865,
    'alb_firn': 0.605,
    'albedo_aging': 13.00,
    'albedo_depth': 8.00,
    'roughness_fresh_snow': 0.81,
    'roughness_ice': 10.35,
    'roughness_firn': 4.05,
    'aging_factor_roughness': 0.0026
}

# set up helper functions

def process_data(df, var, ref_row, x_min, x_max):
    """
    Processes a dataframe slice: sorts it, calculates normalized 'Std'
    using a global range, and calculates the change in mass balance 'PercMB'.
    """
    ref_mb_val = df.loc[ref_row]['mb']
    df_copy = df.sort_values(by=var).reset_index(drop=True)
    if (x_max - x_min) > 0:
        df_copy['Std'] = (df_copy[var] - x_min) / (x_max - x_min)
    else:
        df_copy['Std'] = 0.5 # Assign center value if range is zero
    df_copy['PercMB'] = df_copy['mb'] - ref_mb_val
    return df_copy

def plot_special_rrr_factor(df, axis, axis2):
    """
    Handles the special plotting logic for the rrr_factor subplot only.
    """
    var = 'rrr_factor'
    color_map = {0.5: 'peru', 1.25: 'lightseagreen', 2.0: 'steelblue'}
    
    axis.plot(df['Std'], df['PercMB'], marker='^', linestyle="--", color="black", zorder=1)
    axis2.plot(df['Std'], df['logpALB'], marker='P', linestyle="--", color="black", zorder=1)
    
    for pf, c in color_map.items():
        subset = df.loc[df[var] == pf]
        axis.plot(subset['Std'], subset['PercMB'], marker='^', color=c, linestyle='None', zorder=2)
        axis2.plot(subset['Std'], subset['logpALB'], marker='P', color=c, linestyle='None', zorder=2)

def create_plot_object_mb_alb(df, axis, axis2, color='peru', marker_mb='^', marker_alb='P'):
    """
    Plots a single line for Delta MB and Log-Prob Albedo for standard parameters.
    """
    axis.plot(df['Std'], df['PercMB'], marker=marker_mb, linestyle="--", color=color)
    axis2.plot(df['Std'], df['logpALB'], marker=marker_alb, linestyle="--", color=color)

# Setup Plot
fig, ax = plt.subplots(2, 5, figsize=(20, 12), dpi=300, sharey=True)
ax2_array = np.array([[axis.twinx() for axis in row] for row in ax])

xticks_map = {
    'rrr_factor':             np.arange(0,2+0.5,0.5),
    'alb_ice':                np.arange(0.1,0.46+0.03*3,0.03*3),
    'alb_snow':               np.arange(0.75,0.98+0.025556*2,0.025556*2),
    'alb_firn':               np.arange(0.46,0.75+0.03222*2,0.03222*2),
    'albedo_aging':           np.arange(1, 25+2.666666*2,2.666666*2),
    'albedo_depth':           np.arange(1,15+1.555555*2,1.555555*2),
    'roughness_ice':          np.arange(0.7,20.0+2.1444444*2,2.1444444*2),
    'roughness_fresh_snow':   np.arange(0.02,1.6+0.17555*2,0.17555*2),
    'roughness_firn':         np.arange(1.6,6.5+0.544444*2,0.544444*2),
    'aging_factor_roughness': np.arange(0.0013,0.0039+0.00026*2,0.00026*2),
}
param_map = {
    'rrr_factor':             {'pos': (0, 0), 'rows': None},
    'alb_ice':                {'pos': (0, 1), 'rows': [0] + list(range(1, 11))},
    'alb_snow':               {'pos': (0, 2), 'rows': [0] + list(range(11, 21))},
    'alb_firn':               {'pos': (0, 3), 'rows': [0] + list(range(21, 31))},
    'albedo_aging':           {'pos': (0, 4), 'rows': [0] + list(range(31, 41))},
    'albedo_depth':           {'pos': (1, 0), 'rows': [0] + list(range(41, 51))},
    'roughness_fresh_snow':   {'pos': (1, 2), 'rows': [0] + list(range(51, 61))},
    'roughness_ice':          {'pos': (1, 1), 'rows': [0] + list(range(61, 71))},
    'roughness_firn':         {'pos': (1, 3), 'rows': [0] + list(range(71, 81))},
    'aging_factor_roughness': {'pos': (1, 4), 'rows': [0] + list(range(81, 91))},
}
precip_factors = [0.5, 1.25, 2.0]
colors = ['peru', 'lightseagreen', 'steelblue']

# Handle the special rrr_factor plot
x_min_rrr, x_max_rrr = prec_df['rrr_factor'].min(), prec_df['rrr_factor'].max()
data_df_rrr = process_data(prec_df, 'rrr_factor', ref_row=5, x_min=x_min_rrr, x_max=x_max_rrr)
plot_special_rrr_factor(data_df_rrr, ax[0, 0], ax2_array[0, 0])

# Loop through all other standard parameters
other_params = [p for p in param_map.keys() if p != "rrr_factor"]
for param in other_params:
    info = param_map[param]
    i, j = info['pos']
    x_min, x_max = df[param].min(), df[param].max()
    
    for factor, color in zip(precip_factors, colors):
        sub_df = df.loc[df['rrr_factor'] == factor].reset_index(drop=True)
        data_slice = sub_df.iloc[info['rows']]
        processed_df = process_data(data_slice, param, ref_row=0, x_min=x_min, x_max=x_max) 
        create_plot_object_mb_alb(processed_df, ax[i,j], ax2_array[i,j], color=color)

# Final decorations
for param, info in param_map.items():
    i, j = info['pos']
    axis = ax[i, j]
    
    axis.set_xlim(0, 1)
    
    # Get the correct data range
    x_min, x_max = (prec_df[param].min(), prec_df[param].max()) if param == "rrr_factor" else (df[param].min(), df[param].max())
    
    axis.set_title(dic_label[param])
    axis.yaxis.offsetText.set_fontsize(14)

    def custom_ticks(x, mn=x_min, mx=x_max):
            #convert back to original values
            orig_max = mx
            orig_min = mn
            return x * (orig_max - orig_min) + orig_min 

    ax2x = axis.secondary_xaxis(-0.15, functions=(custom_ticks, custom_ticks))
    new_ticks = xticks_map[param]
    ax2x.set_xticks(new_ticks)
    if param == "aging_factor_roughness":
        ax2x.set_xticklabels([round(x,5) for x in ax2x.get_xticks()], rotation=35)
    else:
        ax2x.set_xticklabels([round(x,2) for x in ax2x.get_xticks()], rotation=35)
    ax2x.set_xlabel("")
    ref_val = lookup_zero[param]
    if (x_max - x_min) > 0:
        # Manually calculate the normalized position for the reference line
        std_ref = (ref_val - x_min) / (x_max - x_min)
        axis.axvline(x=std_ref, linestyle="--", color="black", zorder=-1, alpha=0.7)
        axis.axhline(y=0, linestyle="--", color="black", zorder=-1, alpha=0.7)

    axis.set_ylabel(r"$\Delta$ MB (m w.e.)") if j == 0 else None
    ax2_array[i, j].set_ylabel(r"$\mathcal{L}(\bar{\alpha}|\theta, X)$") if j == 4 else None
    if j != 4: ax2_array[i, j].tick_params(labelright=False)
        
# All the annotations
list_vals = [1.25, 0.28, 0.865, 0.605, 13.00, 8.00, 10.35, 0.81, 4.05, 0.0026]
k = 0

for i in range(ax.shape[0]):
    for j in range(ax.shape[1]):
        if i == 0:
            if j in [2,3]:
                ax[i,j].annotate(list_vals[k], xy=(0.74, 0.01), xycoords="axes fraction", alpha=0.7, fontsize=16)
            else:
                ax[i,j].annotate(list_vals[k], xy=(0.79, 0.01), xycoords="axes fraction", alpha=0.7, fontsize=16)
        else:
            if j == 0:
                ax[i,j].annotate(list_vals[k], xy=(0.84, 0.01), xycoords="axes fraction", alpha=0.7, fontsize=16)
            elif j == 1:
                ax[i,j].annotate(list_vals[k], xy=(0.74, 0.06), xycoords="axes fraction", alpha=0.7, fontsize=16)
            elif j == 2:
                ax[i,j].annotate(list_vals[k], xy=(0.79, 0.08), xycoords="axes fraction", alpha=0.7, fontsize=16)
            elif j == 3:
                ax[i,j].annotate(list_vals[k], xy=(0.79, 0.06), xycoords="axes fraction", alpha=0.7, fontsize=16)
            else:
                ax[i,j].annotate(list_vals[k], xy=(0.69, 0.08), xycoords="axes fraction", alpha=0.7, fontsize=16)
        k += 1
        
# Adjust plot space and create legend
fig.subplots_adjust(hspace=0.5, wspace=0.3)

mb_handle = mlines.Line2D([], [], color='black', marker='^', linestyle='None', markersize=10, label=r'$\Delta$ MB')
alb_handle = mlines.Line2D([], [], color='black', marker='P', linestyle='None', markersize=10, label=r'$\mathcal{L}(ALB|\theta)$')
color_handles = [mpatches.Patch(color=c, label=f'Prcp. Factor {f}') for c, f in zip(colors, precip_factors)]

all_handles = [mb_handle, alb_handle] + color_handles
fig.legend(handles=all_handles, loc="upper center", bbox_to_anchor=(0.5, 0.01), ncol=5)

fig.subplots_adjust(bottom=0.14)
if 'win' in sys.platform:
    plt.savefig("E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS04_local_sensitivites_mb.png", bbox_inches="tight")
else:
    plt.savefig("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS04_local_sensitivites_mb.png", bbox_inches="tight")

In [20]:
## re-create process_data function
def process_data(df, var, ref_row=0):
    df_copy = df.sort_values(by=var)
    df_copy['Std'] = (df_copy[var] - df_copy[var].min()) / (df_copy[var].max() - df_copy[var].min())
    df_copy['PercMB'] = df_copy['mb'] - df_copy.loc[ref_row]['mb']
    
    return df_copy

## parse into plot function to call and create figure
def create_plot_object_logp(df, var, xticks, yticks, axis, ylabel=True, ylabel2=True, 
                            color="steelblue", marker_mb='o', marker_tsla='s'):
    if var == "rrr_factor":
            axis.plot(df['Std'], df['logpMB'], marker="o", linestyle="--", color="black")
            color_map = {0.5: 'peru', 1.25: 'lightseagreen', 2.0: 'steelblue'}
            for pf, c in color_map.items():
                subset = df.loc[df['rrr_factor'] == pf]
                axis.plot(subset['Std'], subset['logpMB'], marker=marker_mb, color=c)
            
            ax2 = axis.twinx()
            ax2.plot(df['Std'], df['logpTSL'], marker=marker_tsla, linestyle="--", color="black")
            for pf, c in color_map.items():
                subset = df.loc[df['rrr_factor'] == pf]
                ax2.plot(subset['Std'], subset['logpTSL'], marker=marker_tsla, color=c)
            
            # Hide the right y-axis for this specific plot
            ax2.tick_params(labelright=False)
            axis.yaxis.label.set_color('black')
            axis.tick_params(axis='y', colors='black')

    else:
        # Plot MB data
        axis.plot(df['Std'], df['logpMB'], marker=marker_mb, linestyle="--", color=color)
        
        ax2 = axis.twinx()
        # Plot TSLA data
        ax2.plot(df['Std'], df['logpTSL'], marker=marker_tsla, linestyle="--", color=color)
        
        axis.yaxis.label.set_color('black')
        axis.tick_params(axis='y', colors='black')
        ax2.yaxis.label.set_color('black')
        ax2.tick_params(axis='y', colors='black')
        
        if axis not in [ax[0,4], ax[1,4]]:
             ax2.tick_params(labelright=False)

    # formatting
    try:
        zero_point = df.loc[df['PercMB'] == 0.0]
        axis.axvline(x=zero_point['Std'].values, linestyle="--", color="black", zorder=-1, alpha=0.7)
    except:
        std_ref = lookup_zero[var]
        zero_point = df.loc[df[var] == std_ref]
        axis.axvline(x=zero_point['Std'].values, linestyle="--", color="black", zorder=-1, alpha=0.7)
    
    if ylabel:
        axis.set_ylabel(r'$\mathcal{L}(B_{geod}|\theta,X)$')
    if ylabel2:
        ax2.set_ylabel(r'$\mathcal{L}(SLA|\theta,X)$')

    ax2.yaxis.offsetText.set_fontsize(14)
    ax2.set_ylim(-305, 5)

    def custom_ticks(x):
        orig_max = df[var].max()
        orig_min = df[var].min()
        return x * (orig_max - orig_min) + orig_min

    ax2x = axis.secondary_xaxis(-0.1, functions=(custom_ticks, custom_ticks))
    ax2x.set_xticks(xticks)
    
    if var == "aging_factor_roughness":
        ax2x.set_xticklabels([round(x,5) for x in ax2x.get_xticks()], rotation=35)
    else:
        ax2x.set_xticklabels([round(x,2) for x in ax2x.get_xticks()], rotation=35)
        
    axis.set_title(dic_label[var])
    axis.set_xlim(0, 1)

        

In [ ]:
## are the logp snowlines correct?
print(np.nanmax(df.iloc[[0] + list(range(11,21))].std()['sim1':'sim58']))
print(np.nanmax(df.iloc[0:11].std()['sim1':'sim58']))

In [ ]:
testing = process_data(sub.iloc[[0] + list(range(41,51))], var="albedo_depth", ref_row=0)
eval_mb = geod_mb['dmdtda'].values
sigma_mb= geod_mb['err_dmdtda'].values
print(eval_mb, sigma_mb)

loglike_mb = -0.5 * (np.log(2 * np.pi * sigma_mb**2) + ( ((eval_mb-testing.mb)**2) / sigma_mb**2))
loglike_mb

In [ ]:
fig, ax = plt.subplots(2,5, figsize=(20,12), dpi=300, sharey=True)
## Prcp factor all versions and first for prcp factor = 0.5
sub = df.loc[df['rrr_factor'] == 0.5].reset_index(drop=True)
create_plot_object_logp(process_data(prec_df, var="rrr_factor", ref_row=5), "rrr_factor", np.arange(0,2+0.5,0.5), np.arange(-6, 4+2, 2), axis=ax[0,0], ylabel=True, ylabel2=False)
#0.5: 'peru', 1.25: 'lightseagreen', 2.0: 'steelblue'
create_plot_object_logp(process_data(sub.iloc[[0] + list(range(1,11))], var="alb_ice", ref_row=0), "alb_ice", np.arange(0.1,0.46+0.03*2,0.03*2), np.arange(-0.06, 0.06+0.04, 0.04), axis=ax[0,1], ylabel=False, ylabel2=False, color="peru") # COLOR & LABEL CHANGE
create_plot_object_logp(process_data(sub.iloc[[0] + list(range(11,21))], var="alb_snow", ref_row=0), "alb_snow", np.arange(0.75,0.98+0.025556*2,0.025556*2), np.arange(-1.5, 1.5+0.5, 0.5), axis=ax[0,2], ylabel=False, ylabel2=False, color="peru") # COLOR & LABEL CHANGE
create_plot_object_logp(process_data(sub.iloc[[0] + list(range(21,31))], var="alb_firn", ref_row=0), "alb_firn", np.arange(0.46,0.75+0.03222*2,0.03222*2), np.arange(-0.45, 0.45+0.15, 0.15), axis=ax[0,3], ylabel=False, ylabel2=False, color="peru") # COLOR & LABEL CHANGE
create_plot_object_logp(process_data(sub.iloc[[0] + list(range(31,41))], var="albedo_aging", ref_row=0), "albedo_aging", np.arange(1, 25+2.666666*2,2.666666*2), np.arange(-3, 0.5+0.9, 0.9), axis=ax[0,4], ylabel=False, ylabel2=True, color="peru") # COLOR & LABEL CHANGE
create_plot_object_logp(process_data(sub.iloc[[0] + list(range(41,51))], var="albedo_depth", ref_row=0), "albedo_depth", np.arange(1,15+1.555555*2,1.555555*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,0], ylabel=True, ylabel2=False, color="peru") # COLOR & LABEL CHANGE
create_plot_object_logp(process_data(sub.iloc[[0] + list(range(61,71))], var="roughness_ice", ref_row=0), "roughness_ice", np.arange(0.7,20.0+2.1444444*2,2.1444444*2), np.arange(-0.015, 0.005+0.005, 0.005), axis=ax[1,1], ylabel=False, ylabel2=False, color="peru") # COLOR & LABEL CHANGE
create_plot_object_logp(process_data(sub.iloc[[0] + list(range(51,61))], var="roughness_fresh_snow", ref_row=0), "roughness_fresh_snow", np.arange(0.02,1.6+0.17555*2,0.17555*2), np.arange(-0.6, 0.4+0.1, 0.1), axis=ax[1,2], ylabel=False, ylabel2=False, color="peru") # COLOR & LABEL CHANGE
create_plot_object_logp(process_data(sub.iloc[[0] + list(range(71,81))], var="roughness_firn", ref_row=0), "roughness_firn", np.arange(1.6,6.5+0.544444*2,0.544444*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,3], ylabel=False, ylabel2=False, color="peru") # COLOR & LABEL CHANGE
create_plot_object_logp(process_data(sub.iloc[[0] + list(range(81,91))], var="aging_factor_roughness", ref_row=0), "aging_factor_roughness", np.arange(0.0013,0.0039+0.00026*2,0.00026*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,4], ylabel=False, ylabel2=True, color="peru") # COLOR & LABEL CHANGE

#
sub2 = df.loc[df['rrr_factor'] == 1.25].reset_index(drop=True)
#create_plot_object(process_data(prec_df, var="rrr_factor", ref_row=5), "rrr_factor", np.arange(0,2+0.5,0.5), np.arange(-6, 4+2, 2), axis=ax[0,0], ylabel=True, ylabel2=False, color="lightseagreen")
create_plot_object_logp(process_data(sub2.iloc[[0] + list(range(1,11))], var="alb_ice", ref_row=0), "alb_ice", np.arange(0.1,0.46+0.03*2,0.03*2), np.arange(-0.06, 0.06+0.04, 0.04), axis=ax[0,1], ylabel=False, ylabel2=False, color="lightseagreen")
create_plot_object_logp(process_data(sub2.iloc[[0] + list(range(11,21))], var="alb_snow", ref_row=0), "alb_snow", np.arange(0.75,0.98+0.025556*2,0.025556*2), np.arange(-1.5, 1.5+0.5, 0.5), axis=ax[0,2], ylabel=False, ylabel2=False, color="lightseagreen")
create_plot_object_logp(process_data(sub2.iloc[[0] + list(range(21,31))], var="alb_firn", ref_row=0), "alb_firn", np.arange(0.46,0.75+0.03222*2,0.03222*2), np.arange(-0.45, 0.45+0.15, 0.15), axis=ax[0,3], ylabel=False, ylabel2=False, color="lightseagreen")
create_plot_object_logp(process_data(sub2.iloc[[0] + list(range(31,41))], var="albedo_aging", ref_row=0), "albedo_aging", np.arange(1, 25+2.666666*2,2.666666*2), np.arange(-3, 0.5+0.9, 0.9), axis=ax[0,4], ylabel=False, ylabel2=True, color="lightseagreen")
create_plot_object_logp(process_data(sub2.iloc[[0] + list(range(41,51))], var="albedo_depth", ref_row=0), "albedo_depth", np.arange(1,15+1.555555*2,1.555555*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,0], ylabel=True, ylabel2=False, color="lightseagreen")
create_plot_object_logp(process_data(sub2.iloc[[0] + list(range(61,71))], var="roughness_ice", ref_row=0), "roughness_ice", np.arange(0.7,20.0+2.1444444*2,2.1444444*2), np.arange(-0.015, 0.005+0.005, 0.005), axis=ax[1,1], ylabel=False, ylabel2=False, color="lightseagreen")
create_plot_object_logp(process_data(sub2.iloc[[0] + list(range(51,61))], var="roughness_fresh_snow", ref_row=0), "roughness_fresh_snow", np.arange(0.02,1.6+0.17555*2,0.17555*2), np.arange(-0.6, 0.4+0.1, 0.1), axis=ax[1,2], ylabel=False, ylabel2=False, color="lightseagreen")
create_plot_object_logp(process_data(sub2.iloc[[0] + list(range(71,81))], var="roughness_firn", ref_row=0), "roughness_firn", np.arange(1.6,6.5+0.544444*2,0.544444*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,3], ylabel=False, ylabel2=False, color="lightseagreen")
create_plot_object_logp(process_data(sub2.iloc[[0] + list(range(81,91))], var="aging_factor_roughness", ref_row=0), "aging_factor_roughness", np.arange(0.0013,0.0039+0.00026*2,0.00026*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,4], ylabel=False, ylabel2=True, color="lightseagreen")
#
sub3 = df.loc[df['rrr_factor'] == 2.0].reset_index(drop=True)
#create_plot_object(process_data(prec_df, var="rrr_factor", ref_row=5), "rrr_factor", np.arange(0,2+0.5,0.5), np.arange(-6, 4+2, 2), axis=ax[0,0], ylabel=True, ylabel2=False, color="lightseagreen")
create_plot_object_logp(process_data(sub3.iloc[[0] + list(range(1,11))], var="alb_ice", ref_row=0), "alb_ice", np.arange(0.1,0.46+0.03*2,0.03*2), np.arange(-0.06, 0.06+0.04, 0.04), axis=ax[0,1], ylabel=False, ylabel2=False, color="steelblue")
create_plot_object_logp(process_data(sub3.iloc[[0] + list(range(11,21))], var="alb_snow", ref_row=0), "alb_snow", np.arange(0.75,0.98+0.025556*2,0.025556*2), np.arange(-1.5, 1.5+0.5, 0.5), axis=ax[0,2], ylabel=False, ylabel2=False, color="steelblue")
create_plot_object_logp(process_data(sub3.iloc[[0] + list(range(21,31))], var="alb_firn", ref_row=0), "alb_firn", np.arange(0.46,0.75+0.03222*2,0.03222*2), np.arange(-0.45, 0.45+0.15, 0.15), axis=ax[0,3], ylabel=False, ylabel2=False, color="steelblue")
create_plot_object_logp(process_data(sub3.iloc[[0] + list(range(31,41))], var="albedo_aging", ref_row=0), "albedo_aging", np.arange(1, 25+2.666666*2,2.666666*2), np.arange(-3, 0.5+0.9, 0.9), axis=ax[0,4], ylabel=False, ylabel2=True, color="steelblue")
create_plot_object_logp(process_data(sub3.iloc[[0] + list(range(41,51))], var="albedo_depth", ref_row=0), "albedo_depth", np.arange(1,15+1.555555*2,1.555555*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,0], ylabel=True, ylabel2=False, color="steelblue")
create_plot_object_logp(process_data(sub3.iloc[[0] + list(range(61,71))], var="roughness_ice", ref_row=0), "roughness_ice", np.arange(0.7,20.0+2.1444444*2,2.1444444*2), np.arange(-0.015, 0.005+0.005, 0.005), axis=ax[1,1], ylabel=False, ylabel2=False, color="steelblue")
create_plot_object_logp(process_data(sub3.iloc[[0] + list(range(51,61))], var="roughness_fresh_snow", ref_row=0), "roughness_fresh_snow", np.arange(0.02,1.6+0.17555*2,0.17555*2), np.arange(-0.6, 0.4+0.1, 0.1), axis=ax[1,2], ylabel=False, ylabel2=False, color="steelblue")
create_plot_object_logp(process_data(sub3.iloc[[0] + list(range(71,81))], var="roughness_firn", ref_row=0), "roughness_firn", np.arange(1.6,6.5+0.544444*2,0.544444*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,3], ylabel=False, ylabel2=False, color="steelblue")
create_plot_object_logp(process_data(sub3.iloc[[0] + list(range(81,91))], var="aging_factor_roughness", ref_row=0), "aging_factor_roughness", np.arange(0.0013,0.0039+0.00026*2,0.00026*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,4], ylabel=False, ylabel2=True, color="steelblue")
fig.subplots_adjust(hspace=0.44, wspace=0.3)
list_vals = [1.25, 0.28, 0.865, 0.605, 13.00, 8.00, 10.35, 0.81, 4.05, 0.0026]
i = 0
for axis in [ax[0,0], ax[0,1], ax[0,2], ax[0,3], ax[0,4]]:
    if axis == ax[0,1]:
        axis.annotate(list_vals[i], xy=(0.78, 0.01), xycoords="axes fraction", alpha=0.7, fontsize=16)
    elif axis == ax[0,0]:
        axis.annotate(list_vals[i], xy=(0.78, 0.01), xycoords="axes fraction", alpha=0.7, fontsize=16)
    elif axis == ax[0,4]:
        axis.annotate(list_vals[i], xy=(0.78, 0.01), xycoords="axes fraction", alpha=0.7, fontsize=16)
    else:
        axis.annotate(list_vals[i], xy=(0.71, 0.01), xycoords="axes fraction", alpha=0.7, fontsize=16)
    i+= 1
for axis in [ax[1,0], ax[1,1], ax[1,2], ax[1,3], ax[1,4]]:
    if axis == ax[1,1]:
        axis.annotate(list_vals[i], xy=(0.70, 0.95), xycoords="axes fraction", alpha=0.7, fontsize=16)
    elif axis == ax[1,0]:
        axis.annotate(list_vals[i], xy=(0.823, 0.95), xycoords="axes fraction", alpha=0.7, fontsize=16)
    elif axis == ax[1,4]:
        axis.annotate(list_vals[i], xy=(0.632, 0.95), xycoords="axes fraction", alpha=0.7, fontsize=16)
    else:
        axis.annotate(list_vals[i], xy=(0.767, 0.95), xycoords="axes fraction", alpha=0.7, fontsize=16)
    i+= 1
    
# Create legend entries
mb_handle = mlines.Line2D([], [], color='black', marker='o', linestyle='None', markersize=10, label=r'$\mathcal{L}(B_{geod}|\theta,X)$')
tsla_handle = mlines.Line2D([], [], color='black', marker='s', linestyle='None', markersize=10, label=r'$\mathcal{L}(SLA|\theta,X)$')

factor1_handle = mpatches.Patch(color='peru', label='Prcp. Factor 0.5')
factor2_handle = mpatches.Patch(color='lightseagreen', label='Prcp. Factor 1.25')
factor3_handle = mpatches.Patch(color='steelblue', label='Prcp. Factor 2.0')

all_handles = [mb_handle, tsla_handle, factor1_handle, factor2_handle, factor3_handle]
fig.legend(handles=all_handles, loc="upper center", bbox_to_anchor=(0.5, -0.05), ncol=5)

fig.subplots_adjust(bottom=0.06)

if 'win' in sys.platform:
    plt.savefig("E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS05_local_sensitivites_logps.png", bbox_inches="tight")
else:
    plt.savefig("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS05_local_sensitivites_logps.png", bbox_inches="tight")

In [24]:
## Repeat plot but for melt energy

def create_plot_object_me(df, var, xticks, yticks, axis, axis2, ylabel=True, ylabel2=True,
                          color="darkred", marker_abl='D', marker_ann='X'):
    if var == "rrr_factor":
            color_map = {0.5: 'peru', 1.25: 'lightseagreen', 2.0: 'steelblue'}
            
            # Plot MEabl
            axis.plot(df['Std'], df['MEabl'], marker=marker_abl, linestyle="--", color="black", zorder=1) # Added marker
            for pf, c in color_map.items():
                subset = df.loc[df['rrr_factor'] == pf]
                axis.plot(subset['Std'], subset['MEabl'], marker=marker_abl, color=c, linestyle='None', zorder=2)

            # Plot MEann
            axis2.plot(df['Std'], df['MEann'], marker=marker_ann, linestyle="--", color="black", zorder=1) # Added marker
            for pf, c in color_map.items():
                subset = df.loc[df['rrr_factor'] == pf]
                axis2.plot(subset['Std'], subset['MEann'], marker=marker_ann, color=c, linestyle='None', zorder=2)
                
            axis2.tick_params(labelright=False)

    # Standard handling for all other subplots
    else:
        axis.plot(df['Std'], df['MEabl'], marker=marker_abl, linestyle="--", color=color)
        axis2.plot(df['Std'], df['MEann'], marker=marker_ann, linestyle="--", color=color)

    # formatting
    try:
        zero_point = df.loc[df['PercMB'] == 0.0]
        axis.axvline(x=zero_point['Std'].values, linestyle="--", color="black", zorder=-1, alpha=0.7)
    except:
        std_ref = lookup_zero[var]
        zero_point = df.loc[df[var] == std_ref]
        axis.axvline(x=zero_point['Std'].values, linestyle="--", color="black", zorder=-1, alpha=0.7)
    
    if ylabel:
        axis.set_ylabel(r"Abl. season (Wm$^{-2}$)")
    if ylabel2:
        axis2.set_ylabel(r"Annual (Wm$^{-2}$)")
    
    axis.yaxis.label.set_color('black')
    axis.tick_params(axis='y', colors='black')
    axis2.yaxis.label.set_color('black')
    axis2.tick_params(axis='y', colors='black')
    axis.yaxis.offsetText.set_fontsize(14)
    axis2.yaxis.offsetText.set_fontsize(14)

    def custom_ticks(x):
        orig_max = df[var].max()
        orig_min = df[var].min()
        return x * (orig_max - orig_min) + orig_min
    
    ax2x = axis.secondary_xaxis(-0.1, functions=(custom_ticks, custom_ticks))
    ax2x.set_xticks(xticks)
    if var == "aging_factor_roughness":
        ax2x.set_xticklabels([round(x, 5) for x in ax2x.get_xticks()], rotation=35)
    else:
        ax2x.set_xticklabels([round(x, 2) for x in ax2x.get_xticks()], rotation=35)
        
    axis.set_title(dic_label[var])
    axis.set_xlim(0, 1)

    if not ylabel2:
        axis2.tick_params(labelright=False)

In [ ]:
process_data(prec_df, var="rrr_factor", ref_row=5)

In [ ]:
fig, ax = plt.subplots(2,5, figsize=(20,12), dpi=300, sharey=True)

ax2_array = np.array([[axis.twinx() for axis in row] for row in ax])
## Prcp factor all versions and first for prcp factor = 0.5
sub = df.loc[df['rrr_factor'] == 0.5].reset_index(drop=True)

create_plot_object_me(process_data(prec_df, var="rrr_factor", ref_row=5), "rrr_factor", np.arange(0,2+0.5,0.5), np.arange(-6, 4+2, 2), axis=ax[0, 0], axis2=ax2_array[0, 0], ylabel=True, ylabel2=False)
create_plot_object_me(process_data(sub.iloc[[0] + list(range(1,11))], var="alb_ice", ref_row=0), "alb_ice", np.arange(0.1,0.46+0.03*2,0.03*2), np.arange(-0.06, 0.06+0.04, 0.04), axis=ax[0,1], axis2=ax2_array[0,1], ylabel=False, ylabel2=False, color="peru")
create_plot_object_me(process_data(sub.iloc[[0] + list(range(11,21))], var="alb_snow", ref_row=0), "alb_snow", np.arange(0.75,0.98+0.025556*2,0.025556*2), np.arange(-1.5, 1.5+0.5, 0.5), axis=ax[0,2], axis2=ax2_array[0,2], ylabel=False, ylabel2=False, color="peru")
create_plot_object_me(process_data(sub.iloc[[0] + list(range(21,31))], var="alb_firn", ref_row=0), "alb_firn", np.arange(0.46,0.75+0.03222*2,0.03222*2), np.arange(-0.45, 0.45+0.15, 0.15), axis=ax[0,3], axis2=ax2_array[0,3], ylabel=False, ylabel2=False, color="peru")
create_plot_object_me(process_data(sub.iloc[[0] + list(range(31,41))], var="albedo_aging", ref_row=0), "albedo_aging", np.arange(1, 25+2.666666*2,2.666666*2), np.arange(-3, 0.5+0.9, 0.9), axis=ax[0,4], axis2=ax2_array[0,4], ylabel=False, ylabel2=True, color="peru")
create_plot_object_me(process_data(sub.iloc[[0] + list(range(41,51))], var="albedo_depth", ref_row=0), "albedo_depth", np.arange(1,15+1.555555*2,1.555555*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,0], axis2=ax2_array[1,0], ylabel=True, ylabel2=False, color="peru")
create_plot_object_me(process_data(sub.iloc[[0] + list(range(61,71))], var="roughness_ice", ref_row=0), "roughness_ice", np.arange(0.7,20.0+2.1444444*2,2.1444444*2), np.arange(-0.015, 0.005+0.005, 0.005), axis=ax[1,1], axis2=ax2_array[1,1], ylabel=False, ylabel2=False, color="peru")
create_plot_object_me(process_data(sub.iloc[[0] + list(range(51,61))], var="roughness_fresh_snow", ref_row=0), "roughness_fresh_snow", np.arange(0.02,1.6+0.17555*2,0.17555*2), np.arange(-0.6, 0.4+0.1, 0.1), axis=ax[1,2], axis2=ax2_array[1,2], ylabel=False, ylabel2=False, color="peru")
create_plot_object_me(process_data(sub.iloc[[0] + list(range(71,81))], var="roughness_firn", ref_row=0), "roughness_firn", np.arange(1.6,6.5+0.544444*2,0.544444*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,3], axis2=ax2_array[1,3], ylabel=False, ylabel2=False, color="peru")
create_plot_object_me(process_data(sub.iloc[[0] + list(range(81,91))], var="aging_factor_roughness", ref_row=0), "aging_factor_roughness", np.arange(0.0013,0.0039+0.00026*2,0.00026*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,4], axis2=ax2_array[1,4], ylabel=False, ylabel2=True, color="peru")

#
sub2 = df.loc[df['rrr_factor'] == 1.25].reset_index(drop=True)
#create_plot_object(process_data(prec_df, var="rrr_factor", ref_row=5), "rrr_factor", np.arange(0,2+0.5,0.5), np.arange(-6, 4+2, 2), axis=ax[0,0], ylabel=True, ylabel2=False, color="lightseagreen")
create_plot_object_me(process_data(sub2.iloc[[0] + list(range(1,11))], var="alb_ice", ref_row=0), "alb_ice", np.arange(0.1,0.46+0.03*2,0.03*2), np.arange(-0.06, 0.06+0.04, 0.04), axis=ax[0,1], axis2=ax2_array[0,1], ylabel=False, ylabel2=False, color="lightseagreen")
create_plot_object_me(process_data(sub2.iloc[[0] + list(range(11,21))], var="alb_snow", ref_row=0), "alb_snow", np.arange(0.75,0.98+0.025556*2,0.025556*2), np.arange(-1.5, 1.5+0.5, 0.5), axis=ax[0,2], axis2=ax2_array[0,2], ylabel=False, ylabel2=False, color="lightseagreen")
create_plot_object_me(process_data(sub2.iloc[[0] + list(range(21,31))], var="alb_firn", ref_row=0), "alb_firn", np.arange(0.46,0.75+0.03222*2,0.03222*2), np.arange(-0.45, 0.45+0.15, 0.15), axis=ax[0,3], axis2=ax2_array[0,3], ylabel=False, ylabel2=False, color="lightseagreen")
create_plot_object_me(process_data(sub2.iloc[[0] + list(range(31,41))], var="albedo_aging", ref_row=0), "albedo_aging", np.arange(1, 25+2.666666*2,2.666666*2), np.arange(-3, 0.5+0.9, 0.9), axis=ax[0,4], axis2=ax2_array[0,4], ylabel=False, ylabel2=True, color="lightseagreen")
create_plot_object_me(process_data(sub2.iloc[[0] + list(range(41,51))], var="albedo_depth", ref_row=0), "albedo_depth", np.arange(1,15+1.555555*2,1.555555*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,0], axis2=ax2_array[1,0], ylabel=True, ylabel2=False, color="lightseagreen")
create_plot_object_me(process_data(sub2.iloc[[0] + list(range(61,71))], var="roughness_ice", ref_row=0), "roughness_ice", np.arange(0.7,20.0+2.1444444*2,2.1444444*2), np.arange(-0.015, 0.005+0.005, 0.005), axis=ax[1,1], axis2=ax2_array[1,1], ylabel=False, ylabel2=False, color="lightseagreen")
create_plot_object_me(process_data(sub2.iloc[[0] + list(range(51,61))], var="roughness_fresh_snow", ref_row=0), "roughness_fresh_snow", np.arange(0.02,1.6+0.17555*2,0.17555*2), np.arange(-0.6, 0.4+0.1, 0.1), axis=ax[1,2], axis2=ax2_array[1,2], ylabel=False, ylabel2=False, color="lightseagreen")
create_plot_object_me(process_data(sub2.iloc[[0] + list(range(71,81))], var="roughness_firn", ref_row=0), "roughness_firn", np.arange(1.6,6.5+0.544444*2,0.544444*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,3], axis2=ax2_array[1,3], ylabel=False, ylabel2=False, color="lightseagreen")
create_plot_object_me(process_data(sub2.iloc[[0] + list(range(81,91))], var="aging_factor_roughness", ref_row=0), "aging_factor_roughness", np.arange(0.0013,0.0039+0.00026*2,0.00026*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,4], axis2=ax2_array[1,4], ylabel=False, ylabel2=True, color="lightseagreen")
#
sub3 = df.loc[df['rrr_factor'] == 2.0].reset_index(drop=True)
#create_plot_object(process_data(prec_df, var="rrr_factor", ref_row=5), "rrr_factor", np.arange(0,2+0.5,0.5), np.arange(-6, 4+2, 2), axis=ax[0,0], ylabel=True, ylabel2=False, color="lightseagreen")
create_plot_object_me(process_data(sub3.iloc[[0] + list(range(1,11))], var="alb_ice", ref_row=0), "alb_ice", np.arange(0.1,0.46+0.03*2,0.03*2), np.arange(-0.06, 0.06+0.04, 0.04), axis=ax[0,1], axis2=ax2_array[0,1], ylabel=False, ylabel2=False, color="steelblue")
create_plot_object_me(process_data(sub3.iloc[[0] + list(range(11,21))], var="alb_snow", ref_row=0), "alb_snow", np.arange(0.75,0.98+0.025556*2,0.025556*2), np.arange(-1.5, 1.5+0.5, 0.5), axis=ax[0,2], axis2=ax2_array[0,2], ylabel=False, ylabel2=False, color="steelblue")
create_plot_object_me(process_data(sub3.iloc[[0] + list(range(21,31))], var="alb_firn", ref_row=0), "alb_firn", np.arange(0.46,0.75+0.03222*2,0.03222*2), np.arange(-0.45, 0.45+0.15, 0.15), axis=ax[0,3], axis2=ax2_array[0,3], ylabel=False, ylabel2=False, color="steelblue")
create_plot_object_me(process_data(sub3.iloc[[0] + list(range(31,41))], var="albedo_aging", ref_row=0), "albedo_aging", np.arange(1, 25+2.666666*2,2.666666*2), np.arange(-3, 0.5+0.9, 0.9), axis=ax[0,4], axis2=ax2_array[0,4], ylabel=False, ylabel2=True, color="steelblue")
create_plot_object_me(process_data(sub3.iloc[[0] + list(range(41,51))], var="albedo_depth", ref_row=0), "albedo_depth", np.arange(1,15+1.555555*2,1.555555*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,0], axis2=ax2_array[1,0], ylabel=True, ylabel2=False, color="steelblue")
create_plot_object_me(process_data(sub3.iloc[[0] + list(range(61,71))], var="roughness_ice", ref_row=0), "roughness_ice", np.arange(0.7,20.0+2.1444444*2,2.1444444*2), np.arange(-0.015, 0.005+0.005, 0.005), axis=ax[1,1], axis2=ax2_array[1,1], ylabel=False, ylabel2=False, color="steelblue")
create_plot_object_me(process_data(sub3.iloc[[0] + list(range(51,61))], var="roughness_fresh_snow", ref_row=0), "roughness_fresh_snow", np.arange(0.02,1.6+0.17555*2,0.17555*2), np.arange(-0.6, 0.4+0.1, 0.1), axis=ax[1,2], axis2=ax2_array[1,2], ylabel=False, ylabel2=False, color="steelblue")
create_plot_object_me(process_data(sub3.iloc[[0] + list(range(71,81))], var="roughness_firn", ref_row=0), "roughness_firn", np.arange(1.6,6.5+0.544444*2,0.544444*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,3], axis2=ax2_array[1,3], ylabel=False, ylabel2=False, color="steelblue")
create_plot_object_me(process_data(sub3.iloc[[0] + list(range(81,91))], var="aging_factor_roughness", ref_row=0), "aging_factor_roughness", np.arange(0.0013,0.0039+0.00026*2,0.00026*2), np.arange(-0.025, 0.05+0.025, 0.025), axis=ax[1,4], axis2=ax2_array[1,4], ylabel=False, ylabel2=True, color="steelblue")
fig.subplots_adjust(hspace=0.44, wspace=0.3)

# Create legend
me_abl_handle = mlines.Line2D([], [], color='black', marker='D', linestyle='None', markersize=10, label='Abl. Season')
me_ann_handle = mlines.Line2D([], [], color='black', marker='X', linestyle='None', markersize=10, label='Ann. Mean')
precip_factors_me = [0.5, 1.25, 2.0]
colors_me = ['peru', 'lightseagreen', 'steelblue']
color_handles = [mpatches.Patch(color=c, label=f'Prcp. Factor {f}') for c, f in zip(colors_me, precip_factors_me)]

all_handles = [me_abl_handle, me_ann_handle] + color_handles
fig.legend(handles=all_handles, loc="upper center", bbox_to_anchor=(0.5, -0.05), ncol=5)

fig.subplots_adjust(bottom=0.06)

if 'win' in sys.platform:
    plt.savefig("E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS06_local_sensitivites_meltenergy.png", bbox_inches="tight")
else:
    plt.savefig("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS06_local_sensitivites_meltenergy.png", bbox_inches="tight")
